In [1]:
import os
os.chdir('../')
%pwd

'/home/laptop/projects/kidney_classification'

In [2]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    '''Entity that contains information needed for training'''
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [3]:
# update configuration manager
## make use of the Entity above
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

class ConfigurationManager:
    def __init__(
        self,
        config_yaml_filepath=CONFIG_YAML_FILE_PATH,
        params_yaml_filepath=PARAMS_YAML_FILE_PATH,
        ):
        
        self.config = read_yaml(config_yaml_filepath)
        self.params = read_yaml(params_yaml_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        create_directories([Path(training.root_dir)])
        
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
        )
        
        return training_config

2024-01-07 18:51:34.532050: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 18:51:34.532192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 18:51:34.545998: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-07 18:51:34.591758: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 18:51:35.513218: W tensorflow/compiler/tf2

In [4]:
from tensorflow import keras

In [7]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [11]:
# training component

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        """Load the model that we're going to train"""
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
    
    def train_valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20,
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )
        
        valid_datagenerator = ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

    
    
    @staticmethod
    def save_model(path: Path, model):
        model.save(path)

In [12]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-01-07 19:06:44,766: INFO: common: yaml file config/config.yaml loaded successfully]
[2024-01-07 19:06:44,768: INFO: common: yaml file params.yaml loaded successfully]
[2024-01-07 19:06:44,770: INFO: common: Created directory at artifacts]
[2024-01-07 19:06:44,771: INFO: common: Created directory at artifacts/training]
Found 2489 images belonging to 1 classes.
Found 9957 images belonging to 1 classes.
Epoch 1/3


2024-01-07 19:06:46.170743: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-01-07 19:06:49.126094: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-07 19:06:50.249323: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-07 19:06:52.455223: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be

  1/622 [..............................] - ETA: 2:42:55 - loss: 1.4286 - accuracy: 0.6250

I0000 00:00:1704629221.090624    8941 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


622/622 [==============================] - 175s 256ms/step - loss: 34.4416 - accuracy: 0.4999 - val_loss: 45.8902 - val_accuracy: 1.0000
Epoch 2/3
622/622 [==============================] - 163s 261ms/step - loss: 34.5573 - accuracy: 0.5009 - val_loss: 43.9278 - val_accuracy: 1.0000
Epoch 3/3
622/622 [==============================] - 212s 341ms/step - loss: 34.4115 - accuracy: 0.4996 - val_loss: 37.2179 - val_accuracy: 1.0000


/home/laptop/projects/kidney_classification/kidney_env/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
